# Compare different Geo projections

[ERA5 is in WGS84 which is equivalent to EPSG:4326](https://confluence.ecmwf.int/display/CKB/ERA5%3A+data+documentation#heading-Spatialreferencesystems).

[USWTDB provides coordinats in the NAD 83 format](https://eerscmap.usgs.gov/uswtdb/assets/data/uswtdb_v5_1_20220729.xml) (see link to latest [metadata xml here](https://eerscmap.usgs.gov/uswtdb/data/)).

We did not reproject coordinates, but this should not be a problem as differences are very small as shown in this notebook.

In [1]:
from init import *

In [2]:
import geopandas

In [3]:
matplotlib.rc('figure', figsize=(17, 14))

In [4]:
turbines = load_turbines()

In [5]:
points = geopandas.GeoSeries(geopandas.points_from_xy(turbines.xlong, turbines.ylat))

In [6]:
points = points.set_crs('NAD 83')

In [7]:
points_wsg84 = points.to_crs("EPSG:4326")

In [8]:
def geolocation_distances(locations1, locations2):
    """Calculate the pairwise distances for geo locations given in lat/long.

    Parameters
    ----------
    locations : np.ndarray
        with shape (N, 2) - in lat/long

    Returns
    -------
    distance matrix in km of shape (N, N) (symmetric, 0. entries in diagonal)

    """
    # FIXME do we need to take care about different coordinate systems or so?
    # FIXME this is not very heavily tested, not sure about correctness, numerical stability etc
    # TODO performance can be improved at least by factor2, ATM it calculates the full (symmetric)
    #  matrix for each element

    # TODO use sklearn instead? seems to support haversine since DBSCAN can do it

    # FIXME should we use something else instead of Haversine?
    #  --> https://en.wikipedia.org/wiki/Vincenty%27s_formulae

    locations_rad1 = np.radians(locations1)
    latitudes1, longitudes1 = locations_rad1.T
    
    locations_rad2 = np.radians(locations2)
    latitudes2, longitudes2 = locations_rad2.T

    a = (np.sin((latitudes2 - latitudes1)/2)**2 + np.cos(latitudes1) *
         np.cos(latitudes2) * np.sin((longitudes2 - longitudes1)/2)**2)
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

    return EARTH_RADIUS_KM * c

# Average earth radius, see https://en.wikipedia.org/wiki/Earth_radius
EARTH_RADIUS_KM = 6371.0088

Maximum difference between the correctly reprojected points and the ones in NAD 83 are less than 3m apart:

In [9]:
(geolocation_distances(
    np.array([points_wsg84.y, points_wsg84.x]).T,
    np.array([points.y, points.x]).T
) * 1e3).max()

2.3145314912233443

In average the distance is only 26cm:

In [10]:
(geolocation_distances(
    np.array([points_wsg84.x, points_wsg84.y]).T,
    np.array([points.x, points.y]).T
) * 1e3).mean()

0.26327226511504265